<a href="https://colab.research.google.com/github/hirossk/0808python/blob/main/0808LLM%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%81%BE%E3%81%97%E3%82%87%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AIによる判定

学習済みモデルのニューラルネットワークを使って画像を読み込んで解析してみましょう。

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import requests
import torch
import torch.nn as nn
from PIL import Image
from torchvision.models import resnet50
from torchvision.transforms import functional as F


In [ ]:
def main():
    # モデルの準備
    model = resnet50(pretrained=True)
    model.eval()

    # ImageNetのラベルの取得
    response = requests.get("https://git.io/JJkYN")
    labels = response.text.split("\n")

    # 画像分類を行う関数を定義
    @torch.no_grad()
    def inference(input_img):
        print("inference")
        img = Image.fromarray(input_img.astype("uint8"), "RGB")
        print("test")

                # # 前処理
        img = F.resize(img, (224, 224))
        img = F.to_tensor(img)
        img = img.unsqueeze(0)
        img = F.normalize(
            img,
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        )

        # # 推論
        output = model(img).squeeze(0)
        probs = nn.functional.softmax(output, dim=0).numpy()

        # # ラベルごとの確率をdictとして返す
        return {labels[i]: float(probs[i]) for i in range(1000)}
        # return sepia_img

    # 入力の形式を画像とする
    inputs = gr.Image(label="Image")
    # 出力はラベル形式で，top5まで表示する
    outputs = gr.Label(num_top_classes=10)
    # サーバーの立ち上げ
    interface = gr.Interface(inference, gr.Image(), outputs=outputs)
    interface.launch()

main()

## 機械学習のデモ

In [39]:
import gradio as gr
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# データ読み込み
df = pd.read_csv('./data/data.csv', encoding='cp932')

# 「種類」フィールドの値が「宅地(土地と建物)」のデータのみ使用
df = df[df['種類'] == "中古マンション等"]

# 「取引価格（総額）」を万円単位に修正
df["取引価格（総額）"] = df["取引価格（総額）"]/10000

# 建築年の元号表記を築年数に変換
df = df.dropna(subset=['建築年'])
# df['年号'] = df['建築年'].str[:2]
df['西暦'] = df['建築年'].str[:4]
# df.loc[df['年号']=='昭和','築年数'] = 2021 - (df['和暦_年'] + 1925)
# df.loc[df['年号']=='平成','築年数'] = 2021 - (df['和暦_年'] + 1988)
df['築年数'] = 2024 - df['西暦'].astype(int)

# 出力変数
y = df["取引価格（総額）"]

# 学習に必要なカラムを抽出(最寄り駅までの時間、広さ、築年数)
x = df.iloc[:,[8,12,31]]

# ランダムフォレスト回帰モデルの初期化と学習
model = RandomForestRegressor()
model.fit(x, y)

# 不動産価格を予測する関数
def predict_price(time, space, age):
    values = np.array([time, space, age]).reshape(1, -1)  # 入力をNumPy配列に変換して形状を整える
    prediction = model.predict(values)  # 予測を行う
    return prediction[0]

# レイアウト作成
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 不動産予測アプリ
        北海道の不動産価格を予測します！
        """
    )
    with gr.Row():
        area_input = gr.Slider(0, 20, label="最寄り駅までの時間")
        rooms_input = gr.Slider(1, 100, label="広さ")
        rate_input = gr.Number(label="築年数")

        value_output = gr.Number(label="取引価格（万円）")

    area_input.change(predict_price, inputs=[area_input, rooms_input, rate_input], outputs=value_output)
    rooms_input.change(predict_price, inputs=[area_input, rooms_input, rate_input], outputs=value_output)
    rate_input.change(predict_price, inputs=[area_input, rooms_input, rate_input], outputs=value_output)
    value_output.change(predict_price, inputs=[area_input, rooms_input, rate_input], outputs=value_output)
    #インタラクティブに結果が反映される
    demo.load(predict_price, inputs=[area_input, rooms_input, rate_input], outputs=value_output)

# インターフェースを起動
demo.launch()

      最寄駅：距離（分）  面積（㎡）  築年数
0             1     55    8
1             1     55   28
2             0     35   47
3             5     50   40
4             5     30   40
...         ...    ...  ...
1261          9     75   32
1263          2     90   29
1270          5     90   32
1282          6     75   19
1323          7     70   34

[735 rows x 3 columns]
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fa2c7da9b4067315ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install transformers accelerate


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("cyberagent/open-calm-7b", device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("cyberagent/open-calm-7b")

inputs = tokenizer("AIによって私達の暮らしは、", return_tensors="pt").to(model.device)
with torch.no_grad():
    tokens = model.generate(
        **inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.pad_token_id,
    )

output = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(output)


config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/42.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

AIによって私達の暮らしは、もっと便利で快適で豊かなものになるはずです。
しかし、ITやIoT技術が発達しても、それはあくまでも道具に過ぎません。そして、その道具を使いこなすのは「人」です。人が持つ感性・感受性をいかに磨き上げるのか。それがこれからの企業にとって、一番重要な戦略ではないでしょうか?そのために、私達は今まで以上に人と


In [ ]:
inputs = tokenizer("AIによって私達の暮らしは、", return_tensors="pt").to(model.device)
with torch.no_grad():
    tokens = model.generate(
        **inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.pad_token_id,
    )

output = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(output)


AIによって私達の暮らしは、ますます便利で快適になってきました。そして今、人工知能を搭載したロボットが身近になりつつあるのをご存知ですか?ここでは身近な生活の中にある、主に製造業などの産業用途としての人工知能の活用事例をご紹介します。.  Pythonで機械学習をしてみたい方向けにPythonとNumPyの基礎から実践まで解説した記事です。データセットを用意する初期設定や分類・回帰・クラスタリングなど、具体的なプログラミング手法に関して説明しています。また、最後に簡単な予測問題や行列分解についても紹介しています。.
Phone:(994) 726-7763 x 8622


In [ ]:
!pip install transformers
!pip install torch torchvision torchaudio
!pip install accelerate
!git clone https://huggingface.co/cyberagent/open-calm-3b


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "cyberagent/open-calm-3b"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# モデルをCPUまたはGPUに明示的にロードするようにautoから変更
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

model.eval()

requests = [
    "北海道の特徴はなんですか？",
]

system_message = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {user_input} ASSISTANT:"

for req in requests:
    input_req = system_message.format(user_input=req)
    input_ids = tokenizer.encode(input_req, return_tensors="pt").to(device)
    tokens = model.generate(
        input_ids,
        max_new_tokens=500,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    out = tokenizer.decode(tokens[0][len(input_ids[0]):], skip_special_tokens=True)
    print("USER:\n" + req)
    print("ASSISTANT:\n" + out)
    print()
    print()
